# Parneet's Whatapp chat analysis using python



This project do the WA chat analysis using pandas, numpy, matplotlib etc libraries...

Firstly there is a need to have the chat .txt file

For that:
Open Whatsapp group chat
Settings
Click on Export Chat
Click on without media(for simplifying)

In [ ]:
!pip install emoji

In [2]:
#importing libraries
from os import path
from PIL import Image
import emoji
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt



In [3]:
import re
import regex
import datetime
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
% matplotlib inline

In [ ]:
! git clone https://github.com/amueller/word_cloud.git
% cd word_cloud
! pip install .

In [5]:
#specifying pattern 
def DateAndTime(s):
    pattern = '^\[([0-9]+)([\/-])([0-9]+)([\/-])([0-9]+)[,]? ([0-9]+):([0-9][0-9]):([0-9][0-9])[ ]?(AM|PM|am|pm)?\]'
    out = re.match(pattern, s)
    if out:
        return True
    return False

In [ ]:
DateAndTime("[01/09/20, 10:48:16 AM] ‪+91 99180 56238‬: Most welcome, thank you for being a part of the initiative....")

In [7]:
def ToKnowAuthor(s):
    pattern = [
        '([\w]+):',                        # First Name pattern
        '([\w]+[\s]+[\w]+):',              # pattern of First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([\w]+)[\u263a-\U0001f999]+:',    # Name and Emoji  pattern            
    ]
    pattern = '^' + '|'.join(pattern)
    out = re.match(pattern, s)
    if out:
        return True
    return False

In [8]:
def getingInfo(data):   
    splitLine = data.split('] ')
    dateTime = splitLine[0]
    if ',' in dateTime:
      date, time = dateTime.split(',') 
    else:
      date, time = dateTime.split(' ') 
    message = ' '.join(splitLine[1:])
    if ToKnowAuthor(message): 
        splitMessage = message.split(': ') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message


In [ ]:
ChatPath = '/content/_chat.txt' # upload file to notebook 
Data = []
with open(ChatPath, encoding="utf-8") as fp:
    fp.readline() # First line skipped cuz it contains encryption info
    fp.readline()
    fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        data = fp.readline()
        if not data: 
            break 
        data = data.strip()
        if DateAndTime(data): 
            if len(messageBuffer) > 0: 
                Data.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getingInfo(data) 
            messageBuffer.append(message) 
        else:
          data= (data.encode('ascii', 'ignore')).decode("utf-8")
          if DateAndTime(data): 
            if len(messageBuffer) > 0: 
                Data.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getingInfo(data) 
            messageBuffer.append(message) 
          else:
            messageBuffer.append(data)

In [ ]:
def dateconv(date):
  year=''
  if '-' in date:
    year = date.split('-')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d-%m-%Y").strftime("%Y-%m-%d")
    elif len(year) ==2:
      return datetime.datetime.strptime(date, "[%d-%m-%y").strftime("%Y-%m-%d")
  elif '/' in date:
    year = date.split('/')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d/%m/%Y").strftime("%Y-%m-%d")
    if len(year) ==2:
      return datetime.datetime.strptime(date, "[%d/%m/%y").strftime("%Y-%m-%d")
df = pd.DataFrame(Data, columns=['Date', 'Time', 'Author', 'Text']) 
df["Date"] = df["Date"].apply(dateconv)
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
df.head(10)

## telling the senders in the given chat

In [ ]:
df.Author.unique()

Removing the messages created by None

In [ ]:
df = df.dropna()


In [ ]:
df.Author.unique()

Done :)

Analysing the chat in detail!

In [ ]:
total_texts = df.shape[0]
print(total_texts)

To see media texts...


In [ ]:
media_texts = df[df['Text'] == 'image omitted'].shape[0]
print(media_texts)
# in our case 0 as we have excluded media texts in our chat file...

In [ ]:
def count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

df["emoji"] = df["Text"].apply(count)

In [ ]:
emojis = sum(df['emoji'].str.len())
print(emojis)

In [ ]:
print("Analysis")
print("Texts:",total_texts)
print("Media:",media_texts)
print("Emojis:",emojis)

## Getting info abt media, text  and sticker messages in chat file

In [ ]:
media_texts_df = df[(df['Text'] == 'image omitted')|(df['Text'] == 'video omitted')]

In [ ]:
sticker_texts_df = df[df['Text'] == 'sticker omitted']
sticker_texts_df

In [ ]:
texts_df = df.drop(media_texts_df.index)
texts_df = texts_df.drop(sticker_texts_df.index)

In [ ]:
texts_df.info()

In [ ]:
texts_df['Letter_Count'] = texts_df['Text'].apply(lambda s : len(s))
texts_df['Word_Count'] = texts_df['Text'].apply(lambda s : len(s.split(' ')))
texts_df["TextCount"]=1

In [ ]:
texts_df.tail(10)

In [ ]:
texts_df["emojicount"]= df['emoji'].str.len()

#Participants texting analysis

In [ ]:
l = texts_df.Author.unique()
for i in range(len(l)):
  req_df= texts_df[texts_df["Author"] == l[i]]
  print(f'Stats of {l[i]} -')
  print('Total Messages Sent', req_df.shape[0])
  words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
  print('Words per message', words_per_message)
  stickers = sticker_texts_df[sticker_texts_df['Author'] == l[i]].shape[0]
  print('Sticker Messages Sent', stickers)
  media = media_texts_df[media_texts_df['Author'] == l[i]].shape[0]
  print('Media Messages Sent', media)
  emojis = sum(req_df['emoji'].str.len())
  print('Emojis Sent', emojis) 
  print()

# Lets have an analysis on emojis' usage

In [ ]:
total_emojis_list = list(set([a for b in texts_df.emoji for a in b]))
total_emojis = len(total_emojis_list)
print(total_emojis)

### Most used emoji

In [ ]:
total_emojis_list = list([a for b in texts_df.emoji for a in b])
emoji_dict = dict(Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
print(emoji_dict)

In [ ]:
emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
emoji_df

### Emoji distribution visualisation on pie chart😁😁

In [ ]:
import plotly.express as px
fig = px.pie(emoji_df, values='count', names='emoji')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
l = texts_df.Author.unique()
for i in range(len(l)):
  fa_df = texts_df[texts_df['Author'] == l[i]]
  total_emojis_list = list([a for b in fa_df.emoji for a in b])
  emoji_dict = dict(Counter(total_emojis_list))
  emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
  print('Emoji Distribution for', l[i])
  author_emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
  fig = px.pie(author_emoji_df, values='count', names='emoji')
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

More analysis

Day wise 

In [ ]:
def f(i):
  l = [ "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
  return l[i];
day_df=pd.DataFrame(texts_df["Text"])
day_df['day_of_date'] = texts_df['Date'].dt.weekday
day_df['day_of_date'] = day_df["day_of_date"].apply(f)
day_df["messagecount"] = 1
day = day_df.groupby("day_of_date").sum()
day.reset_index(inplace=True)

In [ ]:
fig = px.line_polar(day, r='messagecount', theta='day_of_date', line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    )),
  showlegend=False
)
fig.show()

### Text with max words

In [ ]:
texts_df.iloc[texts_df['Word_Count'].argmax()]

### **Trying Word Cloud**

In [ ]:
Words = " ".join(review for review in texts_df.Text)
print ("There are {} words in all the messages.".format(len(Words)))

In [ ]:
  Endwords = set(STOPWORDS)
  Endwords.update([ "na", "ani","ha","la","eh","ne", "em", "ki", "ah","le","ni","lo","Ma","Haa","ni","ra", "ga"])
  wordcloud = WordCloud(stopwords=Endwords, background_color="white").generate(Words)
  plt.figure( figsize=(10,5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()